In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from keras.utils import np_utils

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [5]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.3 MB/s eta 0:00:00


In [6]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
#from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [7]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [8]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [9]:
Fs = 100

In [ ]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None)

In [10]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/transfer_set_50_subs.csv',header=None)

In [ ]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [ ]:
x_transfer_set=transfer_df.drop([3000],axis=1)

In [ ]:
x_transfer_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.026093,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.032245,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.004273,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.079087,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143


In [ ]:
y_transfer=transfer_df[3000]

In [ ]:
y_transfer

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
y_transfer.value_counts()

2    49030
0    30515
4    19065
3     9095
1     9013
Name: 3000, dtype: int64

In [ ]:
x_transfer_set_rshp=np.array(x_transfer_set).reshape(116718,3000,1)

In [ ]:
x_transfer_set_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [11]:
from keras.utils import np_utils

In [ ]:
from keras.utils import np_utils
y_transfer_=np_utils.to_categorical(y_transfer)

In [ ]:
y_transfer_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
subject_473_tst_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [12]:
subject_473_tst_df=pd.read_csv('/content/gdrive/My Drive/sub_473/x_test_sub.csv',header=None)

In [ ]:
subject_473_tst_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,-0.045043,-0.060749,-0.036475,-0.148801,-0.090258,-0.033144,-0.089782,-0.091686,-0.106441,-0.140709,...,0.018735,-0.010774,-0.001731,0.049197,0.061095,0.037774,0.085845,0.038250,0.019211,0
1,0.047095,0.051678,0.048011,0.055344,0.047095,0.053969,0.051219,0.007224,0.011807,0.089257,...,0.085591,-0.123845,0.060843,-0.138510,-0.149050,0.010890,-0.095431,-0.128428,0.000808,0
2,0.039304,0.029680,0.024639,0.030597,0.038387,0.036554,0.050761,0.059468,0.050761,0.046178,...,-0.126136,-0.138968,-0.127053,-0.122928,-0.116512,-0.115137,-0.115596,-0.124761,-0.114679,0
3,-0.050278,-0.048850,-0.049326,-0.055038,-0.052658,-0.051230,-0.035999,-0.028384,-0.012678,-0.006966,...,-0.013154,-0.013630,-0.010298,-0.006966,-0.005062,-0.004110,-0.013154,-0.029812,-0.042663,1
4,-0.077558,-0.068851,-0.067934,-0.068851,-0.067934,-0.072059,-0.071142,-0.068851,-0.071142,-0.060602,...,-0.026231,-0.032188,-0.032188,-0.040437,-0.044562,-0.038604,-0.036313,-0.038146,-0.016607,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,-0.021721,-0.017437,-0.023625,-0.030288,-0.035999,-0.023625,-0.003634,0.014452,0.014928,0.011120,...,-0.001255,0.003505,-0.003634,-0.007918,-0.013154,-0.012678,-0.020769,-0.018389,-0.014105,2
993,0.015015,0.013640,0.007224,0.016848,0.022806,0.027847,0.031971,0.036554,0.024181,0.009516,...,0.032430,0.026472,0.007682,0.004474,0.023722,0.042512,0.027847,0.027847,0.005391,0
994,-0.006983,-0.001483,-0.001942,-0.007899,0.000350,0.002183,0.001266,-0.006983,0.000350,0.004933,...,0.007682,0.011349,0.020973,0.013640,0.007224,-0.001025,-0.006983,-0.011107,-0.010191,2
995,0.261571,0.240490,0.223076,0.205202,0.196953,0.173123,0.160291,0.153416,0.144251,0.119045,...,0.040679,0.059468,0.064968,0.064510,0.073675,0.075967,0.070926,0.055344,0.041137,4


In [13]:
x_sub_set=subject_473_tst_df.drop([3000],axis=1)

In [14]:
x_sub_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.045043,-0.060749,-0.036475,-0.148801,-0.090258,-0.033144,-0.089782,-0.091686,-0.106441,-0.140709,...,0.025399,0.018735,-0.010774,-0.001731,0.049197,0.061095,0.037774,0.085845,0.038250,0.019211
1,0.047095,0.051678,0.048011,0.055344,0.047095,0.053969,0.051219,0.007224,0.011807,0.089257,...,-0.113763,0.085591,-0.123845,0.060843,-0.138510,-0.149050,0.010890,-0.095431,-0.128428,0.000808
2,0.039304,0.029680,0.024639,0.030597,0.038387,0.036554,0.050761,0.059468,0.050761,0.046178,...,-0.147676,-0.126136,-0.138968,-0.127053,-0.122928,-0.116512,-0.115137,-0.115596,-0.124761,-0.114679
3,-0.050278,-0.048850,-0.049326,-0.055038,-0.052658,-0.051230,-0.035999,-0.028384,-0.012678,-0.006966,...,-0.003159,-0.013154,-0.013630,-0.010298,-0.006966,-0.005062,-0.004110,-0.013154,-0.029812,-0.042663
4,-0.077558,-0.068851,-0.067934,-0.068851,-0.067934,-0.072059,-0.071142,-0.068851,-0.071142,-0.060602,...,-0.027605,-0.026231,-0.032188,-0.032188,-0.040437,-0.044562,-0.038604,-0.036313,-0.038146,-0.016607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,-0.021721,-0.017437,-0.023625,-0.030288,-0.035999,-0.023625,-0.003634,0.014452,0.014928,0.011120,...,0.004933,-0.001255,0.003505,-0.003634,-0.007918,-0.013154,-0.012678,-0.020769,-0.018389,-0.014105
993,0.015015,0.013640,0.007224,0.016848,0.022806,0.027847,0.031971,0.036554,0.024181,0.009516,...,0.017306,0.032430,0.026472,0.007682,0.004474,0.023722,0.042512,0.027847,0.027847,0.005391
994,-0.006983,-0.001483,-0.001942,-0.007899,0.000350,0.002183,0.001266,-0.006983,0.000350,0.004933,...,0.029222,0.007682,0.011349,0.020973,0.013640,0.007224,-0.001025,-0.006983,-0.011107,-0.010191
995,0.261571,0.240490,0.223076,0.205202,0.196953,0.173123,0.160291,0.153416,0.144251,0.119045,...,0.034263,0.040679,0.059468,0.064968,0.064510,0.073675,0.075967,0.070926,0.055344,0.041137


In [15]:
x_sub_set_rshp=np.array(x_sub_set).reshape(997,3000,1)

In [ ]:
x_sub_set_rshp.shape

(997, 3000, 1)

In [16]:
y_sub_dt=subject_473_tst_df[3000]

In [ ]:
y_sub_dt

0      0
1      0
2      0
3      1
4      2
      ..
992    2
993    0
994    2
995    4
996    0
Name: 3000, Length: 997, dtype: int64

In [ ]:
y_sub_dt.value_counts()

0    632
2    175
1    127
4     63
Name: 3000, dtype: int64

In [17]:
y_sub_dt_=np_utils.to_categorical(y_sub_dt)

In [ ]:
y_sub_dt_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [18]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-c4x8x1yb
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-c4x8x1yb
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=67b1014a8f10ffec144549ebd101fd3d9887c1fe40c4fedb419f785a3a6c5ebb
  Stored in directory: /tmp/pip-ephem-wheel-cache-mgja41ao/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [19]:
from keras_contrib.layers import CRF

In [20]:
from keras import optimizers, losses, activations, models
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from keras_contrib.layers import CRF

WINDOW_SIZE = 100

In [21]:
nclass = 5

In [22]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)

    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)

    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)

    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

Checking for individual model accuracies wrt to the test set

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
loaded_model_shuf2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
preds_m1 = loaded_model_shuf1.predict(x_sub_set_rshp)
preds_m1

array([[9.9958032e-01, 2.8036567e-04, 9.1961374e-05, 1.8273682e-05,
        2.9077128e-05],
       [9.8198223e-01, 1.3998158e-02, 1.1515839e-03, 2.3185379e-05,
        2.8448163e-03],
       [1.8715052e-02, 5.7852644e-01, 3.5142761e-01, 9.4940813e-05,
        5.1235992e-02],
       ...,
       [1.6164240e-04, 2.9320989e-02, 9.6542269e-01, 8.9350367e-05,
        5.0053257e-03],
       [9.8375427e-03, 4.0797561e-02, 2.1762948e-02, 8.8055429e-05,
        9.2751390e-01],
       [9.9713290e-01, 2.1728161e-03, 1.4829446e-04, 5.5812805e-05,
        4.9024413e-04]], dtype=float32)

In [ ]:
preds_m2 = loaded_model_shuf2.predict(x_sub_set_rshp)
preds_m2

array([[9.99702275e-01, 1.80548377e-04, 8.71606535e-05, 2.10115268e-05,
        8.99685529e-06],
       [9.69326854e-01, 2.13843267e-02, 4.82979976e-03, 2.53490307e-05,
        4.43361141e-03],
       [1.06695751e-02, 6.07752383e-01, 2.62038916e-01, 3.19265746e-05,
        1.19507127e-01],
       ...,
       [1.69374267e-04, 3.43278721e-02, 9.60861385e-01, 6.14606397e-05,
        4.57990495e-03],
       [5.41984849e-03, 7.34846368e-02, 2.60180086e-02, 4.38195020e-05,
        8.95033658e-01],
       [9.96980011e-01, 2.45635770e-03, 1.14568415e-04, 1.06788590e-04,
        3.42148851e-04]], dtype=float32)

In [ ]:
preds_m3 = loaded_model_shuf3.predict(x_sub_set_rshp)
preds_m3

array([[9.9962878e-01, 2.3513568e-04, 8.2272396e-05, 2.3160139e-05,
        3.0635507e-05],
       [9.8070383e-01, 1.7120715e-02, 4.3846213e-04, 1.4977973e-05,
        1.7219201e-03],
       [1.7934540e-02, 3.8777775e-01, 4.9216667e-01, 4.3083386e-05,
        1.0207795e-01],
       ...,
       [2.0825621e-04, 4.2319871e-02, 9.5133483e-01, 8.2669394e-05,
        6.0544033e-03],
       [1.4434717e-02, 8.3724856e-02, 4.7570374e-02, 4.1917636e-05,
        8.5422808e-01],
       [9.9155957e-01, 5.5859708e-03, 3.5952186e-04, 8.1800499e-05,
        2.4131057e-03]], dtype=float32)

In [ ]:
preds_m1_fin=np.argmax(preds_m1,axis=1)
preds_m2_fin=np.argmax(preds_m2,axis=1)
preds_m3_fin=np.argmax(preds_m3,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m1 = confusion_matrix(y_sub_dt, preds_m1_fin)
matrix_m1.diagonal()/matrix_m1.sum(axis=1)

array([0.91613924, 0.17322835, 0.88571429,        nan, 0.55555556])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m2 = confusion_matrix(y_sub_dt, preds_m2_fin)
matrix_m2.diagonal()/matrix_m2.sum(axis=1)

array([0.91455696, 0.20472441, 0.84571429, 0.6984127 ])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m3 = confusion_matrix(y_sub_dt, preds_m3_fin)
matrix_m3.diagonal()/matrix_m3.sum(axis=1)

array([0.92879747, 0.11811024, 0.86857143, 0.57142857])

In [23]:
from sklearn.metrics import classification_report

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m1 = f1_score(y_sub_dt, preds_m1_fin, average="macro")
prec_m1 = precision_score(y_sub_dt, preds_m1_fin, average="macro")
recall_m1 = recall_score(y_sub_dt, preds_m1_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m1))
    print("Recall: {}".format(recall_m1))
    print("F1 score: {}".format(f1_m1))

    report_m1 = classification_report(y_sub_dt, preds_m1_fin, digits=6)
    print(report_m1)

Precision: 0.4790909987013109
Recall: 0.5061274856465726
F1 score: 0.4711756454185168
              precision    recall  f1-score   support

           0   0.988055  0.916139  0.950739       632
           1   0.423077  0.173228  0.245810       127
           2   0.695067  0.885714  0.778894       175
           3   0.000000  0.000000  0.000000         0
           4   0.289256  0.555556  0.380435        63

    accuracy                       0.793380       997
   macro avg   0.479091  0.506127  0.471176       997
weighted avg   0.820503  0.793380  0.794743       997



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m2 = f1_score(y_sub_dt, preds_m2_fin, average="macro")
prec_m2 = precision_score(y_sub_dt, preds_m2_fin, average="macro")
recall_m2 = recall_score(y_sub_dt, preds_m2_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m2))
    print("Recall: {}".format(recall_m2))
    print("F1 score: {}".format(f1_m2))

    report_m2 = classification_report(y_sub_dt, preds_m2_fin, digits=6)
    print(report_m2)

Precision: 0.6212569068169653
Recall: 0.6658520889002799
F1 score: 0.6084767216533945
              precision    recall  f1-score   support

           0   0.973064  0.914557  0.942904       632
           1   0.490566  0.204724  0.288889       127
           2   0.711538  0.845714  0.772846       175
           4   0.309859  0.698413  0.429268        63

    accuracy                       0.798395       997
   macro avg   0.621257  0.665852  0.608477       997
weighted avg   0.823790  0.798395  0.797288       997



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m3 = f1_score(y_sub_dt, preds_m3_fin, average="macro")
prec_m3 = precision_score(y_sub_dt, preds_m3_fin, average="macro")
recall_m3 = recall_score(y_sub_dt, preds_m3_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m3))
    print("Recall: {}".format(recall_m3))
    print("F1 score: {}".format(f1_m3))

    report_m3 = classification_report(y_sub_dt, preds_m3_fin, digits=6)
    print(report_m3)

Precision: 0.5689128034745373
Recall: 0.6217269261437257
F1 score: 0.5674563698228651
              precision    recall  f1-score   support

           0   0.973466  0.928797  0.950607       632
           1   0.340909  0.118110  0.175439       127
           2   0.663755  0.868571  0.752475       175
           4   0.297521  0.571429  0.391304        63

    accuracy                       0.792377       997
   macro avg   0.568913  0.621727  0.567456       997
weighted avg   0.795814  0.792377  0.781745       997



In [ ]:
optimizer=Adam(lr=1e-4)

In [24]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [25]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)


    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
m=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
hist_19_cnn = m.fit(x_transfer_set_rshp, y_transfer_, epochs=30)

Epoch 1/30
3648/3648 [==============================] - 31s 5ms/step - loss: 0.7797 - acc: 0.6809
Epoch 2/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.5594 - acc: 0.7686
Epoch 3/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.5156 - acc: 0.7880
Epoch 4/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4893 - acc: 0.7999
Epoch 5/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4658 - acc: 0.8108
Epoch 6/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4493 - acc: 0.8182
Epoch 7/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4297 - acc: 0.8270
Epoch 8/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.4108 - acc: 0.8345
Epoch 9/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.3967 - acc: 0.8403
Epoch 10/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.3855 - acc: 0.8458
Epoch 11/30
3648/36

In [ ]:
accuracy_1d_cnn = m.evaluate(x_sub_set_rshp, y_sub_dt_)

32/32 [==============================] - 0s 3ms/step - loss: 1.4672 - acc: 0.7813


In [ ]:
preds_m = m.predict(x_sub_set_rshp)

In [ ]:
preds_m_fin=np.argmax(preds_m,axis=1)

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m = f1_score(y_sub_dt, preds_m_fin, average="macro")
prec_m = precision_score(y_sub_dt, preds_m_fin, average="macro")
recall_m = recall_score(y_sub_dt, preds_m_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m))
    print("Recall: {}".format(recall_m))
    print("F1 score: {}".format(f1_m))

    report = classification_report(y_sub_dt, preds_m_fin, digits=6)
    print(report)

Precision: 0.46777586658744136
Recall: 0.50400842695296
F1 score: 0.4593405175244475
              precision    recall  f1-score   support

           0   0.958599  0.952532  0.955556       632
           1   0.277778  0.078740  0.122699       127
           2   0.559727  0.937143  0.700855       175
           4   0.075000  0.047619  0.058252        63

    accuracy                       0.781344       997
   macro avg   0.467776  0.504008  0.459341       997
weighted avg   0.746027  0.781344  0.748058       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m = confusion_matrix(y_sub_dt, preds_m_fin)
matrix_m.diagonal()/matrix_m.sum(axis=1)

array([0.95253165, 0.07874016, 0.93714286, 0.04761905])

In [ ]:
cm_m = matrix_m.astype('float') / matrix_m.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_m.diagonal()

array([0.95253165, 0.07874016, 0.93714286, 0.04761905])

# light weight model trained on transfer set + subject specific train set and testing on subject specific test set

In [26]:
subject_473_tr_df=pd.read_csv('/content/gdrive/My Drive/sub_473/x_train_sub.csv',header=None)

In [27]:
subject_473_tr_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.001266,0.002183,0.001725,0.007224,0.008599,0.007224,-0.004233,-0.006066,-0.005608,-0.012482,...,0.041137,0.038387,0.035179,0.031055,0.030138,0.023264,0.012724,0.007224,0.002183,2
1,0.057177,0.074592,0.065426,0.065426,0.043887,0.029680,0.037929,0.037471,0.032888,0.017765,...,-0.022564,-0.022106,-0.025772,-0.009274,-0.043187,-0.033105,-0.030813,-0.028064,-0.040437,0
2,-0.093114,-0.161651,-0.194016,-0.213055,-0.218766,-0.220194,-0.221622,-0.214006,-0.184973,-0.155940,...,0.076326,0.081085,0.082513,0.077278,0.088225,0.088225,0.092508,0.085845,0.088225,0
3,-0.030813,-0.045020,-0.063351,-0.089015,-0.108721,-0.110555,-0.098181,-0.084432,-0.078017,-0.073892,...,0.028763,0.039304,0.039762,0.031513,0.022806,0.033805,0.026930,0.033805,0.040221,4
4,0.043009,0.041581,0.074422,0.055860,0.090129,0.075374,0.056812,0.064903,0.046817,0.075374,...,-0.119767,-0.109772,-0.073600,-0.039331,0.011120,0.073946,0.105359,0.190079,0.207214,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379,2
3984,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632,0
3985,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397,0
3986,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846,1


In [28]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [29]:
transfer_df_plus_sub_tr = transfer_df.append(subject_473_tr_df)

In [30]:
transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379,2
3984,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632,0
3985,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397,0
3986,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846,1


In [31]:
x_transfer_df_plus_sub_tr=transfer_df_plus_sub_tr.drop([3000],axis=1)

In [32]:
x_transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,0.004933,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379
3984,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.022106,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632
3985,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.000567,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397
3986,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.039062,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846


In [33]:
y_transfer_df_plus_sub_tr = transfer_df_plus_sub_tr[3000]

In [34]:
y_transfer_df_plus_sub_tr

0       4
1       3
2       2
3       2
4       0
       ..
3983    2
3984    0
3985    0
3986    1
3987    1
Name: 3000, Length: 120706, dtype: int64

In [35]:
x_transfer_df_plus_sub_tr_rshp=np.array(x_transfer_df_plus_sub_tr).reshape(120706,3000,1)

In [36]:
x_transfer_df_plus_sub_tr_rshp.shape

(120706, 3000, 1)

In [37]:
y_transfer_df_plus_sub_tr_=np_utils.to_categorical(y_transfer_df_plus_sub_tr)

In [38]:
y_transfer_df_plus_sub_tr_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [68]:
light_wt_model=get_model_1d_cnn()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_16 (Conv1D)          (None, 2995, 4)           28        
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 1497, 4)          0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 1493, 4)           84        
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 746, 4)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 743, 10)           170 

In [69]:
hist = light_wt_model.fit(x_transfer_df_plus_sub_tr_rshp, y_transfer_df_plus_sub_tr_, epochs=30)

Epoch 1/30
3773/3773 [==============================] - 18s 4ms/step - loss: 0.8025 - acc: 0.6719
Epoch 2/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.5723 - acc: 0.7649
Epoch 3/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.5302 - acc: 0.7843
Epoch 4/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.5042 - acc: 0.7962
Epoch 5/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.4843 - acc: 0.8055
Epoch 6/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.4700 - acc: 0.8117
Epoch 7/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.4592 - acc: 0.8177
Epoch 8/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.4482 - acc: 0.8215
Epoch 9/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.4364 - acc: 0.8261
Epoch 10/30
3773/3773 [==============================] - 16s 4ms/step - loss: 0.4304 - acc: 0.8296
Epoch 11/30
3773/37

In [70]:
accuracy = light_wt_model.evaluate(x_sub_set_rshp, y_sub_dt_)

32/32 [==============================] - 0s 2ms/step - loss: 0.6407 - acc: 0.7864


In [71]:
preds_light_wt_model = light_wt_model.predict(x_sub_set_rshp)

32/32 [==============================] - 0s 2ms/step


In [72]:
preds_light_wt_model_fin=np.argmax(preds_light_wt_model,axis=1)

In [73]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_light_wt_model = f1_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
prec_light_wt_model = precision_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
recall_light_wt_model = recall_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_light_wt_model))
    print("Recall: {}".format(recall_light_wt_model))
    print("F1 score: {}".format(f1_light_wt_model))

    report = classification_report(y_sub_dt, preds_light_wt_model_fin, digits=6)
    print(report)

Precision: 0.5938100507849632
Recall: 0.6260237940985225
F1 score: 0.557015357781026
              precision    recall  f1-score   support

           0   0.982085  0.954114  0.967897       632
           1   0.382353  0.102362  0.161491       127
           2   0.759494  0.685714  0.720721       175
           4   0.251309  0.761905  0.377953        63

    accuracy                       0.786359       997
   macro avg   0.593810  0.626024  0.557015       997
weighted avg   0.820442  0.786359  0.784511       997



In [74]:
from sklearn.metrics import confusion_matrix
matrix_light_wt_model = confusion_matrix(y_sub_dt, preds_light_wt_model_fin)
matrix_light_wt_model.diagonal()/matrix_light_wt_model.sum(axis=1)

array([0.95411392, 0.1023622 , 0.68571429, 0.76190476])

In [ ]:
import keras

In [ ]:
teacher_model=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_sub_set_rshp, y_sub_dt_)

32/32 [==============================] - 11s 5ms/step - loss: 1.1994 - accuracy: 0.7834


In [ ]:
preds_teacher = teacher_model.predict(x_sub_set_rshp)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
print(classification_report(y_sub_dt, preds_teacher_fin, digits=6))

              precision    recall  f1-score   support

           0   0.991304  0.901899  0.944490       632
           1   0.466667  0.110236  0.178344       127
           2   0.665198  0.862857  0.751244       175
           3   0.000000  0.000000  0.000000         0
           4   0.306667  0.730159  0.431925        63

    accuracy                       0.783350       997
   macro avg   0.485967  0.521030  0.461201       997
weighted avg   0.823973  0.783350  0.780588       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.90189873, 0.11023622, 0.86285714,        nan, 0.73015873])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm

array([[0.90189873, 0.02056962, 0.00474684, 0.00791139, 0.06487342],
       [0.03937008, 0.11023622, 0.46456693, 0.03149606, 0.35433071],
       [0.        , 0.        , 0.86285714, 0.03428571, 0.10285714],
       [       nan,        nan,        nan,        nan,        nan],
       [0.        , 0.04761905, 0.22222222, 0.        , 0.73015873]])

In [ ]:
cm.diagonal()

array([0.90189873, 0.11023622, 0.86285714,        nan, 0.73015873])

In [ ]:
#student_model=get_model_cnn() #issue with unmatched shape
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_transfer_set_rshp, y_transfer_, epochs=40)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/40
3648/3648 [==============================] - 25s 6ms/step - categorical_accuracy: 0.6944 - student_loss: 0.7533 - distillation_loss: 0.0024
Epoch 2/40
3648/3648 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7750 - student_loss: 0.5439 - distillation_loss: 0.0017
Epoch 3/40
3648/3648 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7924 - student_loss: 0.5062 - distillation_loss: 0.0016
Epoch 4/40
3648/3648 [==============================] - 21s 6ms/step - categorical_accuracy: 0.8023 - student_loss: 0.4847 - distillation_loss: 0.0015
Epoch 5/40
3648/3648 [==============================] - 21s 6ms/step - categorical_accuracy: 0.8101 - student_loss: 0.4640 - distillation_loss: 0.0015
Epoch 6/40
3648/3648 [==============================] - 21s 6ms/step - categorical_accuracy: 0.8210 - student_loss: 0.4428 - distillation_loss: 0.0014
Epoch 7/40
3648/3648 [==============================] - 21s 6ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

32/32 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7803 - student_loss: 1.3539


In [ ]:
preds = student_model.predict(x_sub_set_rshp)

In [ ]:
preds

array([[1.0000000e+00, 7.4094072e-15, 6.2719773e-12, 3.9108096e-12,
        1.1036569e-11],
       [9.4125259e-01, 3.1712942e-02, 2.4603133e-04, 1.5807254e-10,
        2.6788447e-02],
       [5.1034046e-03, 1.0493239e-01, 5.6604707e-01, 4.4476396e-06,
        3.2391271e-01],
       ...,
       [4.1487789e-07, 9.4293635e-03, 9.5135212e-01, 1.9667881e-07,
        3.9217908e-02],
       [1.9438231e-02, 3.5701004e-01, 2.1376918e-01, 3.2420946e-06,
        4.0977928e-01],
       [9.9999273e-01, 1.9593360e-08, 5.8202528e-07, 7.6648360e-10,
        6.6411490e-06]], dtype=float32)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
preds_fin

array([0, 0, 2, 2, 2, 0, 4, 0, 0, 2, 0, 4, 4, 0, 0, 0, 2, 2, 2, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 4, 4, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 4, 0,
       2, 0, 0, 0, 0, 0, 2, 0, 0, 4, 2, 0, 0, 0, 0, 2, 1, 4, 0, 0, 4, 2,
       0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 2, 4, 0, 0, 4, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0,
       0, 2, 2, 0, 0, 0, 2, 1, 2, 4, 2, 0, 0, 0, 0, 3, 0, 2, 0, 4, 4, 0,
       2, 0, 0, 4, 0, 0, 0, 0, 2, 2, 4, 0, 0, 0, 2, 2, 0, 4, 0, 0, 0, 2,
       0, 4, 4, 0, 2, 0, 4, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0,
       0, 0, 4, 0, 0, 4, 4, 0, 0, 0, 0, 4, 2, 4, 0, 4, 2, 0, 0, 2, 0, 0,
       2, 0, 0, 2, 0, 1, 0, 4, 0, 2, 0, 4, 2, 1, 2, 2, 0, 0, 0, 2, 0, 4,
       0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 4, 0, 2, 0, 0, 2, 0,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 2, 0, 0, 4, 0, 0, 2, 0, 2, 0, 4, 2, 0, 0, 0, 0, 0, 0,
       4, 0, 0, 4, 0, 0, 1, 2, 2, 0, 0, 0, 2, 2, 0,

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [ ]:
print(accuracy_score(y_sub_dt, preds_fin))
#print(precision_score(y_sub_dt, preds_fin))
#print(recall_score(y_sub_dt, preds_fin))
#print(f1_score(y_sub_dt, preds_fin))

0.7803410230692076


In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1 = f1_score(y_sub_dt, preds_fin, average="macro")
prec = precision_score(y_sub_dt, preds_fin, average="macro")
recall = recall_score(y_sub_dt, preds_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec))
    print("Recall: {}".format(recall))
    print("F1 score: {}".format(f1))

    report = classification_report(y_sub_dt, preds_fin, digits=6)
    print(report)

Precision: 0.41433422930520863
Recall: 0.4752998153711799
F1 score: 0.4183423068400888
              precision    recall  f1-score   support

           0   0.970100  0.924051  0.946515       632
           1   0.210526  0.031496  0.054795       127
           2   0.620553  0.897143  0.733645       175
           3   0.000000  0.000000  0.000000         0
           4   0.270492  0.523810  0.356757        63

    accuracy                       0.780341       997
   macro avg   0.414334  0.475300  0.418342       997
weighted avg   0.767781  0.780341  0.758295       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_stu = confusion_matrix(y_sub_dt, preds_fin)
matrix_stu.diagonal()/matrix_stu.sum(axis=1)

array([0.92405063, 0.03149606, 0.89714286,        nan, 0.52380952])

In [ ]:
cm_stu = matrix_stu.astype('float') / matrix_stu.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_stu.diagonal()

array([0.92405063, 0.03149606, 0.89714286,        nan, 0.52380952])

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

32/32 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7803 - student_loss: 1.3539


In [ ]:
#Distiller's performance (student model's prformance)

#alpha=0.9,
#temperature=4

#30 epochs 85.57
#40 epochs 84.85
#50 epochs 83.88
#60 epochs 83.88

#alpha=0.9,
#temperature=5

#30 epochs 85.18
#40 epochs 85.41
#50 epochs 84.01
#60 epochs